In [1]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec

from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

translating es to en
callhome es-en word level configuration
cnn details:
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 9, 'stride': 1, 'pad': 4}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 29, 'stride': 1, 'pad': 14}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 49, 'stride': 1, 'pad': 24}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 69, 'stride': 1, 'pad': 34}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 89, 'stride': 1, 'pad': 44}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 109, 'stride': 1, 'pad': 54}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 129, 'stride': 1, 'pad': 64}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 149, 'stride': 1, 'pad': 74}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 169, 'stride': 1, 'pad': 84}
{'ndim': 1, 'in_channels': 39, 'out_channels': 100, 'ksize': 189, 'stride': 1, 'pad': 94}
vocab size, en=5691, fr=8904


In [2]:
xp = cuda.cupy if gpuid >= 0 else np

In [3]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [ ]:
if gpuid >= 0:
    # print("here")
    cuda.get_device_from_id(gpuid).use()

In [ ]:
%run nmt_trials.py

rnn_in_units 1000
using SGD optimizer
loading data
finished loading data
Starting experiment
cnn_/dev_17394sen_1-2-10layers_512units_k9-199-20_pool90_mini_cnn_word_gru_adam_callhome_es_en_1.log
cnn_/seq2seq_17394sen_1-2-10layers_512units_k9-199-20_pool90_mini_cnn_word_gru_adam_callhome_es_en_1.model
num sentences=13137 and num epochs=0
split data into 50 buckets, each of width=16


100%|██████████| 13137/13137 [00:45<00:00, 285.69it/s]


padding labels
split data into 50 buckets, each of width=16


 75%|███████▌  | 1860/2476 [00:04<00:01, 390.13it/s]

In [ ]:
pr_ids, en_ids = predict(5,50, cat="train", display=True)

In [ ]:
pr_ids, en_ids = predict(4,50, cat="dev", display=True)

In [ ]:
num_sent=5
cat = "train"
b_indx = 3
print("English predictions, bucket={0:d}, num={1:d}:".format(b_indx, num_sent))
pred_ids, loss = batch_training(num_sent,
                       {0:BATCH_SIZE_LOOKUP[cat][b_indx]},
                       buckets_dict[cat][b_indx:b_indx+1],
                       epoch=-1,
                       train=False,
                       shuffle=False)

en_ids = buckets_dict[cat][b_indx]['y'][:num_sent].data.tolist()

if display:
    for p, e in zip(pred_ids, en_ids):
        # for reference, 1st word is GO_ID, no need to display
        display_words(e[1:], "en", "Ref")
        display_words(p, "en", "Hyp")
        print("-"*50)

In [ ]:
i = 0
X = buckets_dict[cat][b_indx]['X_fwd'][i:i+num_sent]
y = buckets_dict[cat][b_indx]['y'][i:i+num_sent]

pred_sents = []
# compute loss
if len(X) > 0:
    with chainer.using_config('train', False):
        p, loss = model.forward(X, y)
        pred_sents.extend(p.tolist())

for p in pred_sents:
    display_words(p, "en", "Hyp")
    print("-"*50)

In [ ]:
pplx_new = compute_pplx(cat="dev", num_sent=NUM_MINI_DEV_SENTENCES)

In [ ]:
start_here(num_training=5000, num_epochs=2)

In [ ]:
X = buckets_dict['dev'][1]['X_fwd'][:20]
y = buckets_dict['dev'][1]['y'][:20]
X.shape, y.shape

In [ ]:
model.reset_state()
model.forward_enc(X)
model.set_decoder_state()
pred_out= model.decode(F.transpose(y)[0])

In [ ]:
print(F.argmax(pred_out, axis=1))
print(F.argmax(F.softmax(pred_out), axis=1))

In [ ]:
ha = Variable(xp.full((batch_size,), GO_ID, dtype=xp.int32))
model.feed_rnn(model.embed_dec(ha), model.rnn_dec).shape

In [ ]:
not xp.all(ha.data == GO_ID)

In [ ]:
xp.all(F.argmax(pred_out, axis=1).data == EOS_ID)

In [ ]:
model.forward(X, y)

In [ ]:
log_train_fil_name, text_fname, dev_fname, test_fname

In [ ]:
%aimport nmt_trials
from nmt_trials import *

In [ ]:
# buckets, bucket_lengths = populate_buckets()

In [ ]:
_, en, sf1 = get_data_item('053.182')
_, en, sf2 = get_data_item('053.183')
xp.isnan(xp.sum(sf))

In [ ]:
sf1.shape, sf2.shape

In [ ]:
_, _, sp1 = get_data_item("041.002", "train")
_, _, sp2 = get_data_item("041.004", "train")
batch_data = [sp1[:100], sp2[:100]]
print(sp1.shape, sp2.shape)

In [ ]:
buck_indx = 23
i = 0
batch_size = 50

pad_size_speech = (buck_indx+1) * SPEECH_BUCKET_WIDTH
pad_size_en = min(bucket_lengths[buck_indx][3], MAX_EN_LEN)

src_lim = pad_size_speech
tar_lim = pad_size_en

sp_files_in_batch = [t[0] for t in buckets[buck_indx][i:i+batch_size]]

# get the next batch of data
batch_data = []
for sp_fil in sp_files_in_batch:
    _, en_ids, speech_feat = get_data_item(sp_fil, cat="train")
    # print(speech_feat.shape, len(en_ids))
    batch_data.append((speech_feat[:pad_size_speech], en_ids[:pad_size_en]))

# compute loss
# loss = model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=True)

In [ ]:
batch_size = len(batch_data)
in_shape_r, in_shape_c = batch_data[0][0].shape

fwd_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
rev_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
decoder_batch = xp.full((batch_size, pad_size_en+2), PAD_ID, dtype=xp.int32)

In [ ]:
for i, (src, tar) in enumerate(batch_data):
    fwd_encoder_batch[i] = model.pad_array(src, src_lim)
    rev_encoder_batch[i] = model.pad_array(xp.flip(src, axis=0), src_lim)

    tar_data = [GO_ID] + tar + [EOS_ID]
    decoder_batch[i] = model.pad_list(tar_data, tar_lim+2, at_start=False)

In [ ]:
train=True

In [ ]:
fwd_encoder_batch = xp.swapaxes(fwd_encoder_batch, 0,1)
rev_encoder_batch = xp.swapaxes(rev_encoder_batch, 0,1)
decoder_batch = xp.swapaxes(decoder_batch, 0,1)

In [ ]:
L_FWD_STATES = model.encode_speech_batch_lstm(fwd_encoder_batch, model.lstm_enc, train=True)

In [ ]:
L_REV_STATES = model.encode_speech_batch_lstm(rev_encoder_batch, model.lstm_rev_enc, train)
# reverse the states to align them with forward encoder
L_REV_STATES = xp.flip(L_REV_STATES, axis=0)

In [ ]:
return_shape = L_FWD_STATES.shape
model.enc_states = F.concat((L_FWD_STATES, L_REV_STATES), axis=2)
model.enc_states = F.swapaxes(model.enc_states, 0, 1)

In [ ]:
buckets_dict['dev']model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=train)

In [ ]:
L_FWD_STATES.shape

In [ ]:
pad_size_speech / 8

In [ ]:
in_size, batch_size, in_dim = fwd_encoder_batch.shape

In [ ]:
print(in_size, batch_size, in_dim)

In [ ]:
speech_feats = xp.load(os.path.join(speech_dir, "train.npz"))

In [ ]:
haha = speech_feats["041.001"]

In [ ]:
haha.shape

In [ ]:
haha = xp.array([[2,-2], [1,-1], [3,-3]], dtype=xp.float32)

In [ ]:
haha

In [ ]:
meh = F.expand_dims(haha.swapaxes(0,1), axis=0)

In [ ]:
meh.shape

In [ ]:
def cnn_out_size(i, p, k):
    o = (i-k) + 2*p + 1
    print("cnn out = {0:d}".format(o))

In [ ]:
k_width = 2
pad = k_width // 2
# pad = k_width - 1
pad = 1

In [ ]:
if gpuid >= 0:
    # print("here")
    cuda.get_device(gpuid).use()
c1 = L.ConvolutionND(ndim=1, in_channels=SPEECH_DIM, out_channels=1, ksize=k_width, pad=pad).to_gpu()

In [ ]:
c1.W.shape

In [ ]:
i = meh.shape[2]
out_shape = (i-k_width) + 2*pad + 1
print(i, out_shape)

In [ ]:
%%timeit -n1 -r2
ha = c1(meh)
print(ha.shape)
print(ha.data)

### test with small arrays

In [ ]:
mini_test = xp.array([[2,-2,-5], [1,-1,-10], [3,-3,-9], [4,-4,-12]], dtype=xp.float32)

In [ ]:
mini_test, mini_test.shape

In [ ]:
mini_var = F.expand_dims(mini_test.swapaxes(0,1), axis=0)

In [ ]:
mini_var.shape

In [ ]:
mini_k = 3; mini_pad = mini_k // 2
minic1 = L.ConvolutionND(ndim=1, in_channels=3, out_channels=5, ksize=2, stride=1, pad=1).to_gpu()

In [ ]:
cnn_out_size(i=4, k=2, p=1)

In [ ]:
print(minic1.W.shape)
print(minic1.W.data)

In [ ]:
one = minic1.W.data[0]
one.shape

In [ ]:
mini_test[0], mini_test[1]

In [ ]:
h1 = one[0][0]*mini_test[0][0]
h2 = one[1][0]*mini_test[0][1]
h3 = one[2][0]*mini_test[0][2]
h4 = one[0][1]*mini_test[1][0]
h5 = one[1][1]*mini_test[1][1]
h6 = one[2][1]*mini_test[1][2]

In [ ]:
sum([h1, h2, h3, h4, h5, h6])

In [ ]:
mini_cnn_out = minic1(mini_var).data

In [ ]:
mini_cnn_out.shape, mini_cnn_out

In [ ]:
F.max_pooling_nd(mini_cnn_out, ksize=2, stride=1, pad=mini_pad).data

In [ ]:
speech_feats = {}
speech_feats['train'] = xp.load(os.path.join(speech_dir, "train.npz"))

In [ ]:
sp_1 = speech_feats['train']['041.004']
print(len(sp_1)), sp_1.shape

In [ ]:
CHAR_LEVEL

In [ ]:
en_words = [w.word for w in text_data['train']['041.004']['en']]
en_symbols = [w.encode() for w in en_words]
en_ids = [w2i["en"].get(w, UNK_ID) for w in en_symbols]

In [ ]:
en_ids_array = xp.array(en_ids, dtype=xp.int32)

In [ ]:
F.pad_sequence([en_ids_array, en_ids_array], length=20)

In [ ]:
pad_width = 16

In [ ]:
sp1_pad = xp.pad(sp_1, ((pad_width - len(sp_1) % pad_width,0),(0,0)), mode='constant')

In [ ]:
len(sp1_pad), len(sp_1) % pad_width, len(sp1_pad) % pad_width, sp1_pad.shape

In [ ]:
sp1_pad.shape

In [ ]:
sp_1[0,:5], sp1_pad[:20,:5]

In [ ]:
sp1_pad.shape

In [ ]:
empty_t = xp.empty(shape=(0,SPEECH_DIM), dtype=xp.float32)

In [ ]:
empty_t = xp.concatenate((empty_t, sp1_pad), axis=0)
print(empty_t.shape)
empty_t = xp.concatenate((empty_t, sp1_pad), axis=0)
print(empty_t.shape)

In [ ]:
sp1_pad_rev = xp.flipud(sp_1)

In [ ]:
sp1_pad_rev[:3,:3], sp1_pad[-3:,:3]

In [ ]:
%run nmt_trials.py

In [ ]:
# buckets_dict['train'] = prepare_data()

In [ ]:
# buckets_dict['train'][0]["X_fwd"].shape

In [ ]:
len(buckets_dict['dev'][1]["X_fwd"])

In [ ]:
t = buckets_dict['dev'][1]['X_fwd'][:20]
t.shape

In [ ]:
out1 = model.forward_cnn(t)

In [ ]:
h = F.rollaxis(out1, axis=2)
print(h.shape), 
print(model.scale)

In [ ]:
cnn_k_widths

In [ ]:
out1[-1,:,-2:]

In [ ]:
model.forward_enc(t)

In [ ]:
hf.shape, hr.shape

In [ ]:
model.enc_states.shape

In [ ]:
model[model.rnn_enc[0]]

In [ ]:
xp.isnan(xp.sum(h.data)) == True

In [ ]:
xp.isnan(xp.sum(model.enc_states.data)) == True

In [ ]:
# model.forward_highway(out1)

In [ ]:
out1 = model.CNN_9(t)
print(out1.shape)
out2 = model.CNN_19(t)
print(out2.shape)

combined_out = F.concat((out1, out2), axis=1)
print(combined_out.shape)

In [ ]:
cnn_out_h = F.max_pooling_nd(combined_out, ksize=9, stride=10, pad=9//2)
print(cnn_out_h.shape)

In [ ]:
highway_test = L.Highway()

In [ ]:
highway_test(combined_out).shape

In [ ]:
model.highway

In [ ]:
MODEL_TYPE == MODEL_CNN

In [ ]:
model.cnn_out_dim

In [ ]:
cnn_k_widths

In [ ]:
cnn_filters, model.max_pool_stride

In [ ]:
model.cnns

In [ ]:
xp.expand_dims(sp_1,axis=0)

In [ ]:
np.prod(buckets[0]["X_fwd"].shape) * buckets[0]["X_fwd"].itemsize

In [ ]:
bucket_data_fname